In [1]:
import sys
import time
from os import listdir
from os.path import isfile, join
import string
import numpy as np
from sklearn.linear_model import LogisticRegression as LR
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import EnglishStemmer
from collections import OrderedDict
import pandas as pd
from nltk.corpus import stopwords

In [11]:
# imports needed and logging
import gzip
import gensim 
import logging
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
def get_file_names(path):
    return [path + "/" + f for f in listdir(path) if isfile(join(path, f))]

In [4]:
def file_to_tokens(file_name):#, min_word_size, stemmer=None):
    with open(file_name) as f:
        file_string = f.read().rstrip()

        return gensim.utils.simple_preprocess(file_string)
        #return file_string

## Train Word Embeddings


In [5]:
def build_data(pos_file_path, neg_file_path ):
    pos_file_names = get_file_names(pos_file_path)
    neg_file_names = get_file_names(neg_file_path)
    # m is the number of training examples
    m_pos = len(pos_file_names)
    m_neg = len(neg_file_names)
    m = m_pos + m_neg
    print(m)
    pos_labels = np.ones(m_pos)
    neg_labels = np.zeros(m_neg)
    y = np.concatenate((pos_labels, neg_labels), axis=0)
    pos_data=pd.DataFrame(index=list(range(m_pos)),columns=['Review'])
    neg_data=pd.DataFrame(index=list(range(m_neg)),columns=['Review'])
    for i in range(m_pos):
        #pos_data.loc[i,'Review']=text_process(file_to_tokens(pos_file_names[i]))
        pos_data.loc[i,'Review']=file_to_tokens(pos_file_names[i])
    pos_data['Label'] = pos_labels
    for i in range(m_neg):
        #neg_data.loc[i,'Review']=text_process(file_to_tokens(neg_file_names[i]))
        neg_data.loc[i,'Review']=file_to_tokens(neg_file_names[i])
    neg_data['Label'] = neg_labels
    data=pd.concat([pos_data,neg_data],axis=0)
    data=data.sample(frac=1).reset_index(drop=True)
    return data



In [6]:
data_small = build_data('review_polarity_data/pos', 'review_polarity_data/neg' )
data_big_part1 = build_data ('aclImdb/train/pos','aclImdb/train/neg')
data_big_part2 = build_data ('aclImdb/test/pos','aclImdb/test/neg')
data_big = pd.concat([data_big_part1,data_big_part2],axis=0)
data_big = data_big.sample(frac=1).reset_index(drop=True)
data_huge = pd.concat([data_small,data_big],axis=0)
data_huge = data_huge.sample(frac=1).reset_index(drop=True)


2000
25000
25000


In [7]:
len(data_huge)

52000

In [9]:
glove_input_file = "glove.6B/glove.6B.200d.txt"
word2vec_output_file = "glove.6B/glove.6B.200d.txt.word2vec"

In [10]:
glove2word2vec(glove_input_file, word2vec_output_file)

2019-02-09 13:01:44,264 : INFO : converting 400000 vectors from glove.6B/glove.6B.200d.txt to glove.6B/glove.6B.200d.txt.word2vec


(400000, 200)

In [12]:
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

2019-02-09 13:03:37,757 : INFO : loading projection weights from glove.6B/glove.6B.200d.txt.word2vec
2019-02-09 13:04:39,062 : INFO : loaded (400000, 200) matrix from glove.6B/glove.6B.200d.txt.word2vec


In [13]:
# build vocabulary and train model
model = gensim.models.Word2Vec(
    data_huge['Review'],
    size=200,
    window=10,
    min_count=2,
    workers=10)
model.train(data_huge['Review'], total_examples=len(data_huge['Review']), epochs=100)

2019-02-09 13:04:48,244 : INFO : collecting all words and their counts
2019-02-09 13:04:48,251 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-02-09 13:04:48,676 : INFO : PROGRESS: at sentence #10000, processed 2378383 words, keeping 53076 word types
2019-02-09 13:04:49,065 : INFO : PROGRESS: at sentence #20000, processed 4774881 words, keeping 70564 word types
2019-02-09 13:04:49,450 : INFO : PROGRESS: at sentence #30000, processed 7165763 words, keeping 83559 word types
2019-02-09 13:04:49,839 : INFO : PROGRESS: at sentence #40000, processed 9560414 words, keeping 93886 word types
2019-02-09 13:04:50,221 : INFO : PROGRESS: at sentence #50000, processed 11940494 words, keeping 102827 word types
2019-02-09 13:04:50,307 : INFO : collected 104557 word types from a corpus of 12431273 raw words and 52000 sentences
2019-02-09 13:04:50,307 : INFO : Loading a fresh vocabulary
2019-02-09 13:04:50,473 : INFO : effective_min_count=2 retains 65356 unique words (62%

2019-02-09 13:05:34,127 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-02-09 13:05:34,144 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-02-09 13:05:34,152 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-02-09 13:05:34,165 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-02-09 13:05:34,175 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-02-09 13:05:34,191 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-02-09 13:05:34,201 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-09 13:05:34,205 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-09 13:05:34,216 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-09 13:05:34,218 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-09 13:05:34,219 : INFO : EPOCH - 3 : training on 12431273 raw words (9481856 effec

2019-02-09 13:06:18,366 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-02-09 13:06:18,375 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-02-09 13:06:18,415 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-02-09 13:06:18,425 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-02-09 13:06:18,433 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-02-09 13:06:18,439 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-09 13:06:18,440 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-09 13:06:18,445 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-09 13:06:18,447 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-09 13:06:18,448 : INFO : EPOCH - 1 : training on 12431273 raw words (9482883 effective words) took 15.1s, 628608 effective words/s
2019-02-09 13:06:19,478 : INFO : EPOCH 2 -

2019-02-09 13:07:01,578 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-09 13:07:01,579 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-09 13:07:01,581 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-09 13:07:01,585 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-09 13:07:01,586 : INFO : EPOCH - 4 : training on 12431273 raw words (9484788 effective words) took 14.6s, 649711 effective words/s
2019-02-09 13:07:02,619 : INFO : EPOCH 5 - PROGRESS: at 6.99% examples, 638621 words/s, in_qsize 19, out_qsize 0
2019-02-09 13:07:03,626 : INFO : EPOCH 5 - PROGRESS: at 14.79% examples, 679945 words/s, in_qsize 20, out_qsize 0
2019-02-09 13:07:04,639 : INFO : EPOCH 5 - PROGRESS: at 22.00% examples, 683799 words/s, in_qsize 19, out_qsize 0
2019-02-09 13:07:05,640 : INFO : EPOCH 5 - PROGRESS: at 28.96% examples, 675620 words/s, in_qsize 19, out_qsize 2
2019-02-09 13:07:06,654 : INFO : EPOCH 5 -

2019-02-09 13:07:46,123 : INFO : EPOCH 8 - PROGRESS: at 6.98% examples, 638955 words/s, in_qsize 20, out_qsize 4
2019-02-09 13:07:47,123 : INFO : EPOCH 8 - PROGRESS: at 13.50% examples, 623256 words/s, in_qsize 19, out_qsize 0
2019-02-09 13:07:48,134 : INFO : EPOCH 8 - PROGRESS: at 20.43% examples, 634251 words/s, in_qsize 17, out_qsize 2
2019-02-09 13:07:49,158 : INFO : EPOCH 8 - PROGRESS: at 27.16% examples, 632433 words/s, in_qsize 19, out_qsize 0
2019-02-09 13:07:50,223 : INFO : EPOCH 8 - PROGRESS: at 33.32% examples, 613732 words/s, in_qsize 20, out_qsize 4
2019-02-09 13:07:51,235 : INFO : EPOCH 8 - PROGRESS: at 40.84% examples, 631397 words/s, in_qsize 20, out_qsize 3
2019-02-09 13:07:52,258 : INFO : EPOCH 8 - PROGRESS: at 48.51% examples, 643161 words/s, in_qsize 18, out_qsize 1
2019-02-09 13:07:53,281 : INFO : EPOCH 8 - PROGRESS: at 54.92% examples, 636445 words/s, in_qsize 17, out_qsize 2
2019-02-09 13:07:54,286 : INFO : EPOCH 8 - PROGRESS: at 62.06% examples, 639174 words/s, 

...

2019-02-09 13:26:29,566 : INFO : EPOCH 88 - PROGRESS: at 80.93% examples, 688104 words/s, in_qsize 19, out_qsize 0
2019-02-09 13:26:30,576 : INFO : EPOCH 88 - PROGRESS: at 88.16% examples, 687600 words/s, in_qsize 16, out_qsize 3
2019-02-09 13:26:31,582 : INFO : EPOCH 88 - PROGRESS: at 95.67% examples, 688028 words/s, in_qsize 20, out_qsize 3
2019-02-09 13:26:32,103 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-02-09 13:26:32,115 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-02-09 13:26:32,121 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-02-09 13:26:32,127 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-02-09 13:26:32,131 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-02-09 13:26:32,132 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-02-09 13:26:32,137 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-09 13:26:3

2019-02-09 13:27:13,502 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-02-09 13:27:13,504 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-02-09 13:27:13,505 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-02-09 13:27:13,507 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-09 13:27:13,520 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-09 13:27:13,522 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-09 13:27:13,529 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-09 13:27:13,530 : INFO : EPOCH - 91 : training on 12431273 raw words (9484184 effective words) took 13.8s, 686620 effective words/s
2019-02-09 13:27:14,558 : INFO : EPOCH 92 - PROGRESS: at 7.07% examples, 649223 words/s, in_qsize 19, out_qsize 0
2019-02-09 13:27:15,564 : INFO : EPOCH 92 - PROGRESS: at 14.53% examples, 670419 words/s, in_qsize 19, out_qs

2019-02-09 13:27:55,112 : INFO : EPOCH - 94 : training on 12431273 raw words (9482013 effective words) took 14.0s, 677031 effective words/s
2019-02-09 13:27:56,127 : INFO : EPOCH 95 - PROGRESS: at 6.97% examples, 650686 words/s, in_qsize 19, out_qsize 0
2019-02-09 13:27:57,135 : INFO : EPOCH 95 - PROGRESS: at 14.30% examples, 663902 words/s, in_qsize 18, out_qsize 1
2019-02-09 13:27:58,153 : INFO : EPOCH 95 - PROGRESS: at 21.77% examples, 679529 words/s, in_qsize 19, out_qsize 0
2019-02-09 13:27:59,153 : INFO : EPOCH 95 - PROGRESS: at 29.11% examples, 681520 words/s, in_qsize 19, out_qsize 3
2019-02-09 13:28:00,182 : INFO : EPOCH 95 - PROGRESS: at 36.42% examples, 680553 words/s, in_qsize 19, out_qsize 0
2019-02-09 13:28:01,209 : INFO : EPOCH 95 - PROGRESS: at 43.90% examples, 685183 words/s, in_qsize 20, out_qsize 0
2019-02-09 13:28:02,234 : INFO : EPOCH 95 - PROGRESS: at 51.08% examples, 681030 words/s, in_qsize 18, out_qsize 1
2019-02-09 13:28:03,237 : INFO : EPOCH 95 - PROGRESS: at

2019-02-09 13:28:41,745 : INFO : EPOCH 98 - PROGRESS: at 36.79% examples, 683722 words/s, in_qsize 18, out_qsize 1
2019-02-09 13:28:42,774 : INFO : EPOCH 98 - PROGRESS: at 43.68% examples, 679191 words/s, in_qsize 19, out_qsize 6
2019-02-09 13:28:43,801 : INFO : EPOCH 98 - PROGRESS: at 51.48% examples, 682908 words/s, in_qsize 20, out_qsize 4
2019-02-09 13:28:44,808 : INFO : EPOCH 98 - PROGRESS: at 59.05% examples, 685530 words/s, in_qsize 19, out_qsize 0
2019-02-09 13:28:45,825 : INFO : EPOCH 98 - PROGRESS: at 66.45% examples, 686845 words/s, in_qsize 20, out_qsize 0
2019-02-09 13:28:46,838 : INFO : EPOCH 98 - PROGRESS: at 73.82% examples, 686898 words/s, in_qsize 19, out_qsize 0
2019-02-09 13:28:47,845 : INFO : EPOCH 98 - PROGRESS: at 81.09% examples, 686709 words/s, in_qsize 20, out_qsize 1
2019-02-09 13:28:48,855 : INFO : EPOCH 98 - PROGRESS: at 88.34% examples, 686399 words/s, in_qsize 20, out_qsize 1
2019-02-09 13:28:49,864 : INFO : EPOCH 98 - PROGRESS: at 96.17% examples, 689449

(948286097, 1243127300)

In [14]:
model.save("enhanced_glove.model")

2019-02-09 13:32:24,873 : INFO : saving Word2Vec object under enhanced_glove.model, separately None
2019-02-09 13:32:24,889 : INFO : storing np array 'vectors' to enhanced_glove.model.wv.vectors.npy
2019-02-09 13:32:25,385 : INFO : not storing attribute vectors_norm
2019-02-09 13:32:25,390 : INFO : storing np array 'syn1neg' to enhanced_glove.model.trainables.syn1neg.npy
2019-02-09 13:32:26,030 : INFO : not storing attribute cum_table
2019-02-09 13:32:26,356 : INFO : saved enhanced_glove.model


In [15]:
model = Word2Vec.load("enhanced_glove.model")

2019-02-09 13:32:43,664 : INFO : loading Word2Vec object from enhanced_glove.model
2019-02-09 13:32:43,790 : INFO : loading wv recursively from enhanced_glove.model.wv.* with mmap=None
2019-02-09 13:32:43,792 : INFO : loading vectors from enhanced_glove.model.wv.vectors.npy with mmap=None
2019-02-09 13:32:43,851 : INFO : setting ignored attribute vectors_norm to None
2019-02-09 13:32:43,852 : INFO : loading vocabulary recursively from enhanced_glove.model.vocabulary.* with mmap=None
2019-02-09 13:32:43,853 : INFO : loading trainables recursively from enhanced_glove.model.trainables.* with mmap=None
2019-02-09 13:32:43,854 : INFO : loading syn1neg from enhanced_glove.model.trainables.syn1neg.npy with mmap=None
2019-02-09 13:32:43,881 : INFO : setting ignored attribute cum_table to None
2019-02-09 13:32:43,882 : INFO : loaded enhanced_glove.model


## NLP_Task : now let's use our pretrained word embedding to accomplish our task

In [35]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression,RidgeClassifier,Lasso
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.metrics import make_scorer,f1_score, auc, roc_auc_score

In [18]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

/home/ahmedbahgat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [19]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.values())

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


In [20]:
data_huge_train, data_huge_test, label_huge_train, label_huge_test = \
train_test_split(data_huge['Review'], data_huge['Label'], test_size=0.2, random_state = 0)
print(len(data_huge_train), len(data_huge_test), len(data_huge_train) + len(data_huge_test))

41600 10400 52000


In [21]:

pipeline1 = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),  # strings to token integer counts
    ('classifier', RidgeClassifier(random_state=0))
    #('classifier', xgb.XGBClassifier(random_state=0))
])



In [23]:
pipeline1.fit(data_huge_train,label_huge_train)

Pipeline(memory=None,
     steps=[('word2vec vectorizer', <__main__.MeanEmbeddingVectorizer object at 0x7f734e7617f0>), ('classifier', RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=0, solver='auto',
        tol=0.001))])

In [24]:
predictions1 = pipeline1.predict(data_huge_test)

In [25]:
print(roc_auc_score(predictions1,label_huge_test))

0.8839663119148804


In [26]:
print(classification_report(predictions1,label_huge_test))

             precision    recall  f1-score   support

        0.0       0.87      0.89      0.88      5129
        1.0       0.89      0.88      0.88      5271

avg / total       0.88      0.88      0.88     10400

